<a href="https://colab.research.google.com/github/karlriis/trajectory-forecasting/blob/main/Edinburgh_NN_over_all_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/crowdbotp/OpenTraj.git

Cloning into 'OpenTraj'...
remote: Enumerating objects: 17725, done.
remote: Counting objects: 100% (1526/1526), done.
remote: Compressing objects: 100% (826/826), done.
remote: Total 17725 (delta 807), reused 1355 (delta 683), pack-reused 16199
Receiving objects: 100% (17725/17725), 390.77 MiB | 17.95 MiB/s, done.
Resolving deltas: 100% (1245/1245), done.
Checking out files: 100% (15796/15796), done.


In [4]:
! pip install pykalman

     |████████████████████████████████| 228 kB 5.2 MB/s 
  Created wheel for pykalman: filename=pykalman-0.9.5-py3-none-any.whl size=48461 sha256=ad2b4d0cc3ff180c0651368098d8a7f2551fa8d13d745fca7998a947963a4eb2
  Stored in directory: /root/.cache/pip/wheels/6a/04/02/2dda6ea59c66d9e685affc8af3a31ad3a5d87b7311689efce6
Successfully built pykalman


In [5]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.getcwd()) + '/OpenTraj/opentraj/') # Anaconda python can't find the toolkit path without this for some reason

from toolkit.loaders.loader_edinburgh import load_edinburgh
from matplotlib import pyplot as plt
import numpy as np
np.random.seed(42)

In [6]:
opentraj_root = './OpenTraj/'
selected_day = '01Sep' # 3 days of data in total, ['01Jul', '01Aug', '01Sep']
edinburgh_path = os.path.join(opentraj_root, 'datasets/Edinburgh/annotations', 'tracks.%s.txt' % selected_day)
traj_dataset = load_edinburgh(edinburgh_path, title="Edinburgh", 
                              use_kalman=False, scene_id=selected_day, sampling_rate=4)

/content/OpenTraj/opentraj/toolkit/loaders/loader_edinburgh.py:56: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(file, sep="\n|=", header=None,index_col=None)


reading:./OpenTraj/datasets/Edinburgh/annotations/tracks.01Sep.txt


100%|██████████| 2342/2342 [06:00<00:00,  6.49it/s]


Warning! too big dt in [Edinburgh]
finish


In [2]:
data = traj_dataset.data
NUM_OF_STEPS = 5

# Removing any agents which don't have enough steps for model fitting and predicting
agent_ids = data.agent_id.unique()
for agent_id in agent_ids:
    if len(data[data.agent_id == agent_id]) < 2*NUM_OF_STEPS:
        data = data[data.agent_id != agent_id]
agent_ids = data.agent_id.unique()

NameError: ignored

In [6]:
# 'Normalize' the data so that all trajectories will begin at x=0, y=0
for agent_id in agent_ids:
    first_x = data[data.agent_id == agent_id]['pos_x'].iloc[0]
    first_y = data[data.agent_id == agent_id]['pos_y'].iloc[0]
    
    data.loc[data.agent_id == agent_id, 'pos_x'] = data[data.agent_id == agent_id]['pos_x'] - first_x
    data.loc[data.agent_id == agent_id, 'pos_y'] = data[data.agent_id == agent_id]['pos_y'] - first_y

In [7]:
# Select the useful columns from the dataset
data = data[['index', 'agent_id', 'pos_x', 'pos_y']]

In [8]:
from sklearn.model_selection import train_test_split
train_agent_ids, test_agent_ids = train_test_split(agent_ids, test_size=0.2)

In [11]:
# Temporarily use less data to debug NN architecture issues
train_agent_ids = train_agent_ids[:50]
test_agent_ids = test_agent_ids[:10]

### Helper functions

In [12]:
def create_dataset(data, agent_ids):
    dataX, dataY = [], []
    for agent_id in agent_ids:
        x = data[data['agent_id'] == agent_id].iloc[:5][['pos_x', 'pos_y']]
        y = data[data['agent_id'] == agent_id].iloc[5][['pos_x', 'pos_y']]
        dataX.append(x)
        dataY.append(y)
    return np.array(dataX), np.array(dataY)
    
def calculate_error(pred_x, pred_y, test_x, test_y):
    
    # FDE
    final_displacement_x = pred_x[-1] - test_x[-1]
    final_displacement_y = pred_y[-1] - test_y[-1]
    FDE = np.sqrt(final_displacement_x**2 + final_displacement_y**2)
    
    # MSE
    abs_error_x = pred_x - test_x
    abs_error_y = pred_y - test_y
    
    MSE = np.mean(np.square(abs_error_x)) + np.mean(np.square(abs_error_y))
    
    return FDE, MSE

In [13]:
train = create_dataset(data, train_agent_ids)
test = create_dataset(data, test_agent_ids)

In [14]:
from keras.models import Model
from keras.layers import *    

inp = Input((5,2))

'''
x = Dense(64, activation='linear')(inp)

out1 = Dense(1, activation='linear')(x)    
out2 = Dense(1, activation='linear')(x)
'''
x = Dense(16, activation='linear')(inp)
x = Dense(8, activation='linear')(x)

out1 = Dense(1, activation='linear')(x)    
out2 = Dense(1, activation='linear')(x)

model = Model(inputs=inp, outputs=[out1,out2])
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [15]:
sep_y_train_data = np.hsplit(train[1], 2) # separate the y data into pieces to only provide 1 point as NN output
model.fit(train[0], [sep_y_train_data[0], sep_y_train_data[1]], epochs=100, batch_size=2, verbose=2)

Epoch 1/400
25/25 - 4s - loss: 7.2391 - dense_2_loss: 1.8974 - dense_3_loss: 5.3417 - 4s/epoch - 142ms/step
Epoch 2/400
25/25 - 0s - loss: 5.7389 - dense_2_loss: 1.6558 - dense_3_loss: 4.0831 - 53ms/epoch - 2ms/step
Epoch 3/400
25/25 - 0s - loss: 4.6709 - dense_2_loss: 1.4707 - dense_3_loss: 3.2002 - 56ms/epoch - 2ms/step
Epoch 4/400
25/25 - 0s - loss: 3.9906 - dense_2_loss: 1.3697 - dense_3_loss: 2.6209 - 54ms/epoch - 2ms/step
Epoch 5/400
25/25 - 0s - loss: 3.6148 - dense_2_loss: 1.3323 - dense_3_loss: 2.2826 - 55ms/epoch - 2ms/step
Epoch 6/400
25/25 - 0s - loss: 3.4835 - dense_2_loss: 1.3217 - dense_3_loss: 2.1618 - 62ms/epoch - 2ms/step
Epoch 7/400
25/25 - 0s - loss: 3.4236 - dense_2_loss: 1.3126 - dense_3_loss: 2.1110 - 61ms/epoch - 2ms/step
Epoch 8/400
25/25 - 0s - loss: 3.4181 - dense_2_loss: 1.3146 - dense_3_loss: 2.1034 - 64ms/epoch - 3ms/step
Epoch 9/400
25/25 - 0s - loss: 3.4141 - dense_2_loss: 1.3121 - dense_3_loss: 2.1020 - 64ms/epoch - 3ms/step
Epoch 10/400
25/25 - 0s - lo

In [16]:

trainScore = model.evaluate(train[0], [sep_y_train_data[0], sep_y_train_data[1]], verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore[0], np.sqrt(trainScore[0])))

sep_y_test_data = np.hsplit(test[1], 2)
testScore = model.evaluate(test[0], [sep_y_test_data[0], sep_y_test_data[1]], verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore[0], np.sqrt(testScore[0])))

Train Score: 3.40 MSE (1.84 RMSE)
Test Score: 5.97 MSE (2.44 RMSE)


In [17]:
trainPredict = model.predict(train[0])
testPredict = model.predict(test[0])

In [18]:
for agent_id in test_agent_ids[:20]:
  seen_x = data[data['agent_id'] == agent_id].iloc[:5]['pos_x']
  seen_y = data[data['agent_id'] == agent_id].iloc[:5]['pos_y']
  unseen_x = data[data['agent_id'] == agent_id].iloc[5]['pos_x']
  unseen_x = data[data['agent_id'] == agent_id].iloc[5]['pos_x']

  a = np.column_stack((seen_x, seen_y))  
  print(a.shape)
  model.predict(a)
  
  break
  prediction = model.predict()

(5, 2)


ValueError: ignored

In [1]:
model.predict(np.array([[1,2],[1,2],[1,2],[1,2],[1,2]]))

NameError: ignored

In [21]:
print(train[0][0])
print(a)

[[ 0.          0.        ]
 [-0.0221581  -0.49772777]
 [-0.25798153 -1.38352893]
 [-0.45490984 -1.94707063]
 [-0.67965422 -2.5840079 ]]
[[ 0.          0.        ]
 [-0.37636703  0.36862711]
 [-0.94459661  2.49320105]
 [-1.11505365  3.07569682]
 [-1.32784796  3.81003813]]
